# Tutorial 1: Environment Creation and Training

In this tutorial, we will show a simple example of how to create a gym environment using [EAGERx](https://eagerx.readthedocs.io/en/master/).
Also, we will use this environment to train a policy using [Stable Baselines 3](https://stable-baselines3.readthedocs.io/en/master/).

The aim of this tutorial is to show some of the key concepts of EAGERx:
- Creating a [Graph](https://eagerx.readthedocs.io/en/master/guide/api_reference/graph/graph.html) with an [Object](https://eagerx.readthedocs.io/en/master/guide/api_reference/object/index.html)
- How to use this [Graph](https://eagerx.readthedocs.io/en/master/guide/api_reference/graph/graph.html) and a [Bridge](https://eagerx.readthedocs.io/en/master/guide/api_reference/bridge/index.html) to create an [Eagerx Environment](https://eagerx.readthedocs.io/en/master/guide/api_reference/env/index.html)
- How to train a policy with the [EAGERx Environment](https://eagerx.readthedocs.io/en/master/guide/api_reference/env/index.html)

In the remainder of this tutorial we will go more into detail on these concepts.


## Pendulum Swing-up

We will create an environment for solving the classic control problem of swinging up an underactuated pendulum, very similar to the [Pendulum-v0 environment](https://gym.openai.com/envs/Pendulum-v0/).
Our goal is to swing up this pendulum to the upright position and keep it there, while minimizing the velocity of the pendulum and the input voltage.

Since the dynamics of a pendulum actuated by a DC motor are well known, we can simulate the pendulum by integrating the corresponding ordinary differential equations (ODEs):


$\mathbf{x} = \begin{bmatrix} \theta \\ \dot{\theta} \end{bmatrix} \\ \dot{\mathbf{x}} = \begin{bmatrix} \dot{\theta} \\ \frac{1}{J}(\frac{K}{R}u - mgl \sin{\theta} - b \dot{\theta} - \frac{K^2}{R}\dot{\theta})\end{bmatrix}$

with $\theta$ the angle w.r.t. upright position, $\dot{\theta}$ the angular velocity, $u$ the input voltage, $J$ the inertia, $m$ the mass, $g$ the gravitational constant, $l$ the length of the pendulum, $b$ the motor viscous friction constant, $K$ the motor constant and $R$ the electric resistance.

<img src="./figures/pendulum.GIF" width="480" />

## Notebook Setup

In order to be able to run the code, we need to install the *eagerx_tutorials* package and ROS.

In [2]:
try:
    import eagerx_tutorials
except ImportError:
    !{"pip install eagerx-tutorials  >> /tmp/eagerx_install.txt"}
if 'google.colab' in str(get_ipython()):
  !{"curl 'https://raw.githubusercontent.com/eager-dev/eagerx_tutorials/master/scripts/setup_colab.sh' > ~/setup_colab.sh"}
  !{"bash ~/setup_colab.sh"}

# Setup interactive notebook
# Required in interactive notebooks only.
from eagerx_tutorials import helper
helper.setup_notebook()
env = None

# Allows reloading of registered entites from changed files
# Required in interactive notebooks only.
%reload_ext autoreload
%autoreload 1

Not running on CoLab.
Execute ROS commands as "!...".
ROS noetic available.


## Let's get started

First we will import EAGERx and initialize it.
As mentioned before, EAGERx makes use of ROS functionality for communication and during initialization a ROS master is started if there isn't one running already.

In [3]:
import eagerx
# Initialize eagerx (starts roscore if not already started.)
eagerx.initialize("eagerx_core")

... logging to /home/jelle/.ros/log/8a826b06-c474-11ec-b150-67b88a6c22f1/roslaunch-jelle-Alienware-m15-R4-26379.log
started roslaunch server http://145.94.217.36:39153/
ros_comm version 1.15.14


SUMMARY

PARAMETERS
 * /rosdistro: noetic
 * /rosversion: 1.15.14

NODES

auto-starting new master
process[master]: started with pid [26505]
ROS_MASTER_URI=http://localhost:11311
setting /run_id to 8a826b06-c474-11ec-b150-67b88a6c22f1
process[rosout-1]: started with pid [26530]
started core service [/rosout]


An `Object` is an entity that has inputs (sensors), outputs (actuators) and states (that can be reset at the beginning of an episode).

We are going to create one object (the pendulum). For this first tutorial, we don't want to go into details too much and start with an existing object.
If you are interested, you can find its definition [here](https://github.com/eager-dev/eagerx_tutorials/blob/master/eagerx_tutorials/pendulum/objects.py).
Note that we import the pendulum.
While this might look like an unused import, it is not.
During the import, the pendulum object is registered and we can therefore make it based on its ID, i.e. *Pendulum*.

Before making the object, we will first obtain some info on the *Pendulum*, such that we know with what arguments we should make it.

In [4]:
import eagerx_tutorials.pendulum  # Registers Pendulum

eagerx.Object.info("Pendulum")

Registered entity_id=`Pendulum`:
   entity_type: `Object`
   module: `eagerx_tutorials.pendulum.objects`
   file: `/home/jelle/eagerx_dev/eagerx_tutorials/eagerx_tutorials/pendulum/objects.py`

Supported bridges:
 - OdeBridge

Make this spec with (use `entity_id: str = "Pendulum"`):
   spec = Object.make(entity_id: str, name: str, actuators: List[str] = None, sensors: List[str] = None, states: List[str] = None, rate: float = 30.0, render_shape: List[int] = None)

class Pendulum(Object):
   spec(spec: eagerx.core.specs.ObjectSpec, name: str, actuators: List[str] = None, sensors: List[str] = None, states: List[str] = None, rate: float = 30.0, render_shape: List[int] = None):
      docs:
         Object spec of Pendulum

   agnostic(spec: eagerx.core.specs.ObjectSpec, rate: float):
      config:
       - render_shape: [480, 480]
      sensors:
       - theta: <class 'std_msgs.msg._Float32.Float32'>
       - dtheta: <class 'std_msgs.msg._Float32.Float32'>
       - image: <class 'sensor_msg

We see that the `eagerx.Object.info("Pendulum")` provides us information on the *Pendulum* object.
It has four sensors (*theta*, *dtheta*, *image*, *u*), one actuator (*u*) and two states (*model_state*, *model_parameters*).
Here *theta*, *dtheta* and *u* correspond to $\theta$, $\dot{\theta}$ and $u$, respectively.
For now, we are only interested in how to make this object, other information will be covered in later tutorials.
We can make the *Pendulum* object with the `eagerx.Object.make` method with the required arguments *entity_id* and (a unique) *name*.
Furthermore, we will specify which actuators, sensors and states we will use:

In [5]:
# Make pendulum
pendulum = eagerx.Object.make("Pendulum", "pendulum", actuators=["u"], sensors=["theta", "dtheta"], states=["model_state"])

Next, we create a [Graph](https://eagerx.readthedocs.io/en/master/guide/api_reference/graph/graph.html) and add the pendulum to it.

The graph describes the interconnect of nodes and objects.
In this way, the creation of an environment becomes modular.
This allows users to create an implementation for nodes and objects once, and easily create new environments by reusing these implementations.
Also, this allows to construct complex environments using nodes and objects as basic building blocks.

After adding the pendulum to the graph, we will connect the actuator *u* to a new action called *voltage*.
We will connect the sensors *theta* and *dtheta* to the observations *angle* and *angular_velocity*, respectively.
In this way, the agent will be able to send actions to control $u$ of the pendulum and observe $\theta$ and $\dot{\theta}$.

In [6]:
# Define rate (depends on rate of ode)
rate = 30.0

# Initialize empty graph
graph = eagerx.Graph.create()

# Add pendulum to the graph
graph.add(pendulum)

# Connect the pendulum to an action and observation
graph.connect(action="voltage", target=pendulum.actuators.u)
graph.connect(source=pendulum.sensors.theta, observation="angle")
graph.connect(source=pendulum.sensors.dtheta, observation="angular_velocity")

It is also possible to inspect the graph using the eagerx-gui package.
It can be installed as follows:
```bash
pip3 install eagerx-gui
```
Jupyter notebooks have limited support for interactive applications, so we cannot open the GUI here.
But if we were to run
```python
graph.gui()
```
The ouput would be as follows:

<img src="./figures/tutorial_1_gui.svg" width=720>

Here we see that the actions of the agent are outputs of *env/actions* and that the observations of the agent are inputs of *env/observations*.
Also, we could render output by connecting to *env/render*, which will be covered in another tutorial.
Note that *env/actions*, *env/observations* and *env/render* represent connections of the `Graph` to the environment.
They are split up in the GUI as nodes for visualization purposes.

Next, we will create the [Bridge](https://eagerx.readthedocs.io/en/master/guide/api_reference/bridge/index.html).
Since objects can have implementions for multiple physics engines and real systems, we need to initialize the appropriate bridge.
In our case, we will use the [OdeBridge](https://github.com/eager-dev/eagerx_ode), which allows to simulate systems based on ordinary differential equations (ODEs).
In other tutorials we will go more into detail on the bridge and how you can create your own bridge.

In [7]:
import eagerx_ode  # Registers OdeBridge

# Define bridges
bridge = eagerx.Bridge.make("OdeBridge", rate=rate)

Just like in normal Gym environments, we will create a step function in which we will calculate the reward at each time step and check for termination conditions.
Our goal is to stabilize the pendulum in upright position, while minimizing the input voltage that is applied.
Therefore we choose a reward function that is a weighted sum of $\theta^2$, $\dot{\theta^2}$ and $u^2$. 

Note that we can obtain the values of the actions and observations using the keys *voltage*, *angle* and *angular_velocity*, which correspond to the names of the actions and observations above in the screenshot of the GUI.

In [8]:
import numpy as np
from typing import Dict

# Define step function
def step_fn(prev_obs: Dict[str, np.ndarray], obs: Dict[str, np.ndarray], action: Dict[str, np.ndarray], steps: int):
    
    # Get angle and angular velocity
    # Take first element because of window size (covered in other tutorial)
    th = obs["angle"][0] 
    thdot = obs["angular_velocity"][0]
    
    # Convert from numpy array to float
    u = float(action["voltage"])
    
    # Normalize angle so it lies in [-pi, pi]
    th -= 2 * np.pi * np.floor((th + np.pi) / (2 * np.pi))
    
    # Calculate cost
    # Penalize angle error, angular velocity and input voltage
    cost = th**2 + 0.1 * thdot**2 + 0.001 * u**2  
    
    # Determine when is the episode over
    # currently just a timeout after 100 steps
    done = steps > 100
    
    # Set info, tell the algorithm the termination was due to a timeout
    # (the episode was truncated)
    info = {"TimeLimit.truncated": steps > 100}
    
    return obs, -cost, done, info

Having created a graph, a bridge and a step function, we can now construct the EAGERx environment.

In [9]:
# Initialize Environment
env = eagerx.EagerxEnv(name="PendulumEnv", rate=rate, graph=graph, bridge=bridge, step_fn=step_fn)

[INFO] [1650876541.375696]: Node "/PendulumEnv/env/supervisor" initialized.
[INFO] [1650876541.529052]: Node "/PendulumEnv/bridge" initialized.
[INFO] [1650876541.654216]: Node "/PendulumEnv/environment" initialized.
[INFO] [1650876541.696992]: Node "/PendulumEnv/pendulum/theta" initialized.
[INFO] [1650876541.719355]: Node "/PendulumEnv/pendulum/dtheta" initialized.
[INFO] [1650876541.809380]: Adding object "pendulum" of type "Pendulum" to the simulator.
[INFO] [1650876541.823278]: Node "/PendulumEnv/pendulum/x" initialized.
[INFO] [1650876541.840653]: Node "/PendulumEnv/pendulum/image" initialized.
[INFO] [1650876541.853323]: Node "/PendulumEnv/pendulum/pendulum_actuator" initialized.


The environment we have created, can be used like any other Gym environment.
Here we will now train a policy to swing up the pendulum using the Soft Actor Critic (SAC) reinforcement learning algorithm implementation from [Stable Baselines 3](https://stable-baselines3.readthedocs.io/en/master/).

In [10]:
import stable_baselines3 as sb
from eagerx.wrappers import Flatten

# Stable Baselines3 expects flattened actions & observations
# Convert observation and action space from Dict() to Box()
env = Flatten(env)

# Initialize learner
model = sb.SAC("MlpPolicy", env, verbose=1, device="cpu")

# Train for 1 minute (sim time)
model.learn(total_timesteps=int(60 * rate))

env.shutdown()

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
[INFO] [1650876543.728355]: Nodes initialized.
[INFO] [1650876543.772499]: Pipelines initialized.
----------------------------------
| rollout/           |           |
|    ep_len_mean     | 101       |
|    ep_rew_mean     | -1.01e+03 |
| time/              |           |
|    episodes        | 4         |
|    fps             | 89        |
|    time_elapsed    | 4         |
|    total_timesteps | 404       |
| train/             |           |
|    actor_loss      | 18.7      |
|    critic_loss     | 12.6      |
|    ent_coef        | 0.916     |
|    ent_coef_loss   | -0.118    |
|    learning_rate   | 0.0003    |
|    n_updates       | 303       |
----------------------------------
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 101      |
|    ep_rew_mean     | -960     |
| time/              |          |
|    episodes        | 8        |
|    fps 